In [24]:
import requests
import pandas as pd

# Replace with your OpenCage API key
OPENCAGE_API_KEY = "a2c4781446524691bd94caacffb5efac"

def get_coordinates(address):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        "q": address,
        "key": OPENCAGE_API_KEY,
        "limit": 1
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return location['lat'], location['lng']
        else:
            print(f"Address '{address}' not found.")
            return None, None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None, None

def get_weather_data(latitude, longitude):
    url = f"https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": "true",
        "hourly": "temperature_2m,wind_speed_10m",
        "daily": "rain_sum,snowfall_sum",
        "timezone": "auto"
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        current_weather = data.get('current_weather', {})
        daily_weather = data.get('daily', {})

        return {
            "Latitude": latitude,
            "Longitude": longitude,
            "Temperature": current_weather.get("temperature"),
            "Is Day or Night": "Day" if current_weather.get("is_day") == 1 else "Night",
            "Rain": daily_weather.get("rain_sum", [None])[0],
            "Snowfall": daily_weather.get("snowfall_sum", [None])[0],
            "Weather Code": current_weather.get("weathercode"),
            "Wind Speed": current_weather.get("windspeed"),
            "Wind Direction": current_weather.get("winddirection"),
        }
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def fetch_weather_data_for_addresses(file_path):
    # Load the Excel file
    df = pd.read_csv(file_path)

    # Initialize lists to store weather data
    weather_data_list = []

    for index, row in df.iterrows():
        # Concatenate address fields
        address = f"{row['Address']}, {row['City']}, {row['State']} {row['Zip Code']}"
        latitude, longitude = get_coordinates(address)

        if latitude is not None and longitude is not None:
            weather_data = get_weather_data(latitude, longitude)
            if weather_data:
                weather_data_list.append(weather_data)
            else:
                # Append empty data if weather data couldn't be retrieved
                weather_data_list.append({
                    "Latitude": latitude,
                    "Longitude": longitude,
                    "Temperature": None,
                    "Is Day or Night": None,
                    "Rain": None,
                    "Snowfall": None,
                    "Weather Code": None,
                    "Wind Speed": None,
                    "Wind Direction": None,
                })
        else:
            weather_data_list.append({
                "Latitude": None,
                "Longitude": None,
                "Temperature": None,
                "Is Day or Night": None,
                "Rain": None,
                "Snowfall": None,
                "Weather Code": None,
                "Wind Speed": None,
                "Wind Direction": None,
            })

    # Add weather data to the DataFrame
    weather_df = pd.DataFrame(weather_data_list)
    final_df = pd.concat([df, weather_df], axis=1)

    return final_df

# Usage
if __name__ == "__main__":
    file_path = "/Users/akhilpenmatcha/Downloads/walmart_store_data_copy2.csv"
    weather_df = fetch_weather_data_for_addresses(file_path)
    print(weather_df)


Address '2550 COORS BLDV. NW, ALBUQUERQUE, NM 87120-2123' not found.
Address '501 SW 19TH ST, MOORE, OK 73160-5427' not found.
Address '640 SE 4TH ST, MOORE, OK 73160-6733' not found.
Address '100 COLONY LN, LATROBE, PA 15650-9073' not found.
Address '4691 STATE HIGHWAY 121, COLONY, TX 75056-4010' not found.
Address 'W190N9855 APPLETON AVE, GERMANTOWN, WI 53022-4504' not found.
      Store              Address          City State    Zip Code  \
0      1636      3900 S BROADWAY         MINOT    ND  58701-7534   
1      3875     1625 COMMERCE DR      WAHPETON    ND  58075-3168   
2      1565       4001 2ND AVE W     WILLISTON    ND  58801-2603   
3      2784        3620 N 6TH ST      BEATRICE    NE  68310-1141   
4      2847      10504 S 15TH ST      BELLEVUE    NE  68123-4084   
...     ...                  ...           ...   ...         ...   
1941   1412     4308 E GRAND AVE       LARAMIE    WY  82070-5508   
1942   4471      2390 E CEDAR ST       RAWLINS    WY  82301-6026   
1943   

In [25]:
weather_df.head(10)


,Store,Address,City,State,Zip Code,Phone,Latitude,Longitude,Temperature,Is Day or Night,Rain,Snowfall,Weather Code,Wind Speed,Wind Direction
0,1636,3900 S BROADWAY,MINOT,ND,58701-7534,(701) 838-2176,48.194679,-101.300274,-1.6,Night,0.00,0.0,0.0,8.0,188.0
1,3875,1625 COMMERCE DR,WAHPETON,ND,58075-3168,(701) 642-9086,46.336591,-96.792057,-2.5,Night,2.86,0.1,0.0,5.8,277.0
2,1565,4001 2ND AVE W,WILLISTON,ND,58801-2603,(701) 572-8550,48.182032,-103.630364,-1.3,Night,0.00,0.0,0.0,5.7,145.0
3,2784,3620 N 6TH ST,BEATRICE,NE,68310-1141,(402) 228-1244,40.304196,-96.743971,5.2,Night,0.00,0.0,0.0,6.9,152.0
4,2847,10504 S 15TH ST,BELLEVUE,NE,68123-4084,(402) 292-0156,41.156124,-95.936276,4.4,Night,0.00,0.0,0.0,5.0,60.0
5,3154,2109 TOWNE CENTER DR,BELLEVUE,NE,68123-0000,(402) 686-2040,41.116262,-95.942809,4.3,Night,0.00,0.0,0.0,3.2,63.0
6,4568,1882 HOLLY ST,BLAIR,NE,68008-6309,(402) 533-8222,41.525933,-96.140451,2.9,Night,0.00,0.0,0.0,9.0,164.0
7,2579,510 LINDEN ST,CHADRON,NE,69337-6989,(308) 432-6999,42.827383,-103.022119,1.7,Night,0.00,0.0,0.0,5.0,240.0
8,774,818 E 23RD ST,COLUMBUS,NE,68601-3866,(402) 564-1668,41.438542,-97.312494,2.7,Night,0.00,0.0,1.0,6.1,87.0
9,4322,1800 E 29TH ST,CRETE,NE,68333-3074,(402) 826-1002,40.640564,-96.939566,4.6,Night,0.00,0.0,0.0,9.5,169.0


In [26]:
weather_df.to_csv("weather_output_df2.csv", index=False)

In [28]:
file1_df = pd.read_csv("/Users/akhilpenmatcha/Desktop/DE-Project/weather-project/weather_output_df1.csv")
file2_df = pd.read_csv("/Users/akhilpenmatcha/Desktop/DE-Project/weather-project/weather_output_df2.csv")
walmart_weather_df = pd.concat([file1_df, file2_df], ignore_index=True)
walmart_weather_df = walmart_weather_df.drop(['Weather Code'], axis=1)
walmart_weather_df.to_csv('walmart_weather_df_output.csv', index=False)

In [31]:
walmart_weather_df=walmart_weather_df.dropna(subset=['Latitude', 'Longitude', 'Temperature', 'Is Day or Night', 'Rain', 'Snowfall', 'Wind Speed', 'Wind Direction'])
empty_rows = walmart_weather_df[walmart_weather_df[['Latitude', 'Longitude','Temperature', 'Is Day or Night', 'Rain', 'Snowfall', 'Wind Speed', 'Wind Direction']].isnull().any(axis=1)]
print(empty_rows)


Empty DataFrame
Columns: [Store, Address, City, State, Zip Code, Phone, Latitude, Longitude, Temperature, Is Day or Night, Rain, Snowfall, Weather Code, Wind Speed, Wind Direction]
Index: []


In [32]:
import plotly.express as px
import plotly.io as pio

# Ensure plotly is rendered in Jupyter Notebook
pio.renderers.default = "iframe"  # or try "iframe" if "notebook" doesn't work

fig = px.scatter_mapbox(
    walmart_weather_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="Store",
    hover_data={
        "Zip Code": True,
        "City": True,
        "Address": True,
        "State": True,
        
    },
    zoom=3,
    height=600,
    title="Walmart Store Weather Impact - Severity and Urgency Heatmap"
)

# Set the mapbox style
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center={"lat": 54.5260, "lon": -105.2551}  # Center slightly between U.S. and Alaska

    #mapbox_center={"lat": 37.0902, "lon": -95.7129}  # Center of the United States
)

# Show the plot
fig.show()


In [38]:
walmart_weather_df.head()


,Store,Address,City,State,Zip Code,Phone,Latitude,Longitude,Temperature,Is Day or Night,Rain,Snowfall,Wind Speed,Wind Direction
0,2070,3101 A ST,ANCHORAGE,AK,99503-4050,(907) 563-5900,61.192242,-149.879650,-2.0,Night,0.0,6.72,2.3,288.0
1,2071,8900 OLD SEWARD HWY,ANCHORAGE,AK,99515-2022,(907) 344-5300,61.140050,-149.871154,-2.0,Night,0.0,6.72,2.3,288.0
2,4359,7405 DEBARR ROAD,ANCHORAGE,AK,99504-0000,(907) 339-9039,61.212165,-149.743372,-1.9,Night,0.0,4.76,5.2,236.0
3,2188,18600 EAGLE RIVER RD,EAGLE RIVER,AK,99577-8688,(907) 694-9780,61.309083,-149.535545,-2.5,Night,0.0,4.76,5.2,236.0
4,2722,537 JOHANSEN EXPY,FAIRBANKS,AK,99701-3165,(907) 451-9900,64.858595,-147.690644,-7.4,Night,0.0,0.35,8.6,22.0


In [41]:

# Define a function to generate detailed alerts and individual impact levels
def generate_alerts(row):
    impact_levels = []
    alerts = []

    # Temperature-based alerts
    if row['Temperature'] <= -10:
        impact_levels.append("High")
        alerts.append("Extreme cold alert! Close outdoor areas and ensure heating systems are functioning. Protect pipes from freezing.")
    elif -10 < row['Temperature'] <= 0:
        impact_levels.append("Medium")
        alerts.append("Cold weather advisory. Ensure heating and place extra mats at entrances to prevent slips.")
    elif row['Temperature'] >= 35:
        impact_levels.append("Medium")
        alerts.append("Heat warning! Check air conditioning, provide water stations, and monitor refrigerated sections.")
    elif row['Temperature'] >= 40:
        impact_levels.append("High")
        alerts.append("Extreme heat alert! Limit outdoor activities, ensure A/C is operational, and prepare for potential power issues.")
    else:
        impact_levels.append("Low")
        alerts.append("No significant temperature-related risks.")

    # Day or Night + Heavy Rain/Snow
    if row['Is Day or Night'] == "Night":
        if row['Rain'] > 10 or row['Snowfall'] > 5:
            impact_levels.append("Medium")
            alerts.append("Low visibility at night with rain/snow. Increase outdoor lighting and clear entrances.")
        else:
            impact_levels.append("Low")
            alerts.append("No visibility-related risks at night.")
    elif row['Is Day or Night'] == "Day":
        if row['Rain'] > 10 or row['Snowfall'] > 5:
            impact_levels.append("Medium")
            alerts.append("Rain/snow expected during the day. Advise caution for employees and customers entering the store.")
        else:
            impact_levels.append("Low")
            alerts.append("No visibility-related risks during the day.")

    # Rain-based alerts
    if row['Rain'] > 50:
        impact_levels.append("Severe")
        alerts.append("Flood risk alert! Close store if flooding occurs nearby. Place sandbags and secure entrances.")
    elif 20 <= row['Rain'] <= 50:
        impact_levels.append("High")
        alerts.append("Heavy rain warning! Monitor for water buildup. Secure entryways and deploy additional mats.")
    elif row['Rain'] > 10:
        impact_levels.append("Medium")
        alerts.append("Light rain expected. Place mats at entrances and caution employees and customers to prevent slips.")
    else:
        impact_levels.append("Low")
        alerts.append("No significant rain-related risks.")

    # Snowfall-based alerts
    if row['Snowfall'] > 30:
        impact_levels.append("Severe")
        alerts.append("Heavy snowstorm! Consider closing the store or delaying opening. Ensure snow-clearing supplies are available.")
    elif 15 <= row['Snowfall'] <= 30:
        impact_levels.append("High")
        alerts.append("Heavy snow alert! Assign snow clearance teams and increase lighting in parking lots.")
    elif row['Snowfall'] > 5:
        impact_levels.append("Medium")
        alerts.append("Snow expected. Clear paths and entrances, and check salt supplies.")
    else:
        impact_levels.append("Low")
        alerts.append("No significant snowfall-related risks.")


    # Wind speed-based alerts
    if row['Wind Speed'] >= 90:
        impact_levels.append("Severe")
        alerts.append("Extreme wind alert! Close outdoor areas and postpone any outdoor activities. Monitor windows and fragile structures.")
    elif 70 <= row['Wind Speed'] < 90:
        impact_levels.append("High")
        alerts.append("High wind warning! Secure all outdoor equipment and avoid outdoor activities.")
    elif 40 <= row['Wind Speed'] < 70:
        impact_levels.append("Medium")
        alerts.append("Moderate wind warning. Secure loose outdoor items and signage.")
    else:
        impact_levels.append("Low")
        alerts.append("No significant wind-related risks.")

    # Combined conditions for severe scenarios
    if row['Rain'] > 50 and row['Wind Speed'] > 70:
        impact_levels.append("Severe")
        alerts.append("Storm conditions with heavy rain and strong winds. Consider closing store temporarily. Ensure emergency supplies are available.")
    elif row['Snowfall'] > 15 and row['Wind Speed'] > 40:
        impact_levels.append("High")
        alerts.append("Snowstorm with strong winds expected. Limit outdoor activities and ensure snow-clearing teams are ready.")
    else:
        impact_levels.append("Low")
        alerts.append("No combined severe weather risks.")

    # Determine the overall impact level by taking the highest impact level in the row
    level_priority = {"Low": 1, "Medium": 2, "High": 3, "Severe": 4}
    max_impact_level = max(impact_levels, key=lambda x: level_priority[x])

    # Return the maximum impact level and the combined alerts
    return max_impact_level, "\n".join(alerts)

# Apply the function to each row in the DataFrame and store results in two columns
walmart_weather_df[['Impact Level', 'Alerts']] = walmart_weather_df.apply(lambda row: pd.Series(generate_alerts(row)), axis=1)

# Save the updated DataFrame with alerts to a new CSV file
walmart_weather_df.to_csv("walmart_weather_with_alerts.csv", index=False)

# Display the updated DataFrame with Impact Level and Alerts columns for preview
print(walmart_weather_df[['Store', 'Impact Level', 'Alerts']])


      Store Impact Level                                             Alerts
0      2070       Medium  Cold weather advisory. Ensure heating and plac...
1      2071       Medium  Cold weather advisory. Ensure heating and plac...
2      4359       Medium  Cold weather advisory. Ensure heating and plac...
3      2188       Medium  Cold weather advisory. Ensure heating and plac...
4      2722       Medium  Cold weather advisory. Ensure heating and plac...
...     ...          ...                                                ...
4429   1412       Medium  Cold weather advisory. Ensure heating and plac...
4430   4471       Medium  Cold weather advisory. Ensure heating and plac...
4431   1457       Medium  Cold weather advisory. Ensure heating and plac...
4432   1461       Medium  Cold weather advisory. Ensure heating and plac...
4433   1508          Low  No significant temperature-related risks.\nNo ...

[4425 rows x 3 columns]


In [56]:
import plotly.express as px
import plotly.io as pio

# Ensure plotly is rendered in Jupyter Notebook
pio.renderers.default = "iframe"  # or try "iframe" if "notebook" doesn't work

# Mapping severity to values for visualization
impact_level_map = {'Severe': 'severe', 'High': 'high', 'Medium': 'medium', 'Low': 'low'}

# Apply these mappings to the final dataset
walmart_weather_df['Impact Color'] = walmart_weather_df['Impact Level'].map(impact_level_map).fillna('gray')


fig = px.scatter_mapbox(
    walmart_weather_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="Store",
    hover_data={
        "Zip Code": True,
        "City": True,
        "Address": True,
        "State": True,
        "Impact Level": True,
        
    },
    color="Impact Color",  # Set color based on severity
 #   category_orders={"Impact Level": ["Severe", "High", "Medium", "Low"]},  # Force order

 #   color_discrete_map=impact_level_map,  # Force only mapped colors
    zoom=3,
    height=600,
    title="Walmart Store Weather Impact - Severity and Urgency Heatmap"
)

# Set the mapbox style
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center={"lat": 54.5260, "lon": -105.2551}  # Center slightly between U.S. and Alaska

    #mapbox_center={"lat": 37.0902, "lon": -95.7129}  # Center of the United States
)

# Show the plot
fig.show()


In [114]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib

# Multi-target setup: 'Impact Level' and 'Alert' are the target variables
X = walmart_weather_df[['Temperature', 'Rain', 'Snowfall', 'Wind Speed', 'Wind Direction', 'Is Day or Night']]
y = walmart_weather_df[['Impact Level', 'Alerts']]

# Train-test split for multi-output targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode 'Is Day or Night' for the model
label_encoder = LabelEncoder()
X_train['Is Day or Night'] = label_encoder.fit_transform(X_train['Is Day or Night'])
X_test['Is Day or Night'] = label_encoder.transform(X_test['Is Day or Night'])

# Initialize the multi-output classifier with RandomForest
multi_output_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
multi_output_model.fit(X_train, y_train)

# Save the multi-output model
joblib.dump(multi_output_model, 'multi_output_rf_model.pkl')

# Model evaluation
y_pred = multi_output_model.predict(X_test)
accuracy_impact = accuracy_score(y_test['Impact Level'], y_pred[:, 0])
accuracy_alert = accuracy_score(y_test['Alerts'], y_pred[:, 1])

print("Impact Level Model Accuracy:", accuracy_impact)
print("Alert Model Accuracy:", accuracy_alert)

Impact Level Model Accuracy: 0.9977401129943503
Alert Model Accuracy: 0.9977401129943503


In [57]:
# from sklearn.model_selection import train_test_split

# # Assuming 'Impact Level' is the target and the rest are features
# X = walmart_weather_df[['Temperature', 'Rain', 'Snowfall', 'Wind Speed', 'Wind Direction', 'Is Day or Night']]
# y = walmart_weather_df['Impact Level']

# # Perform train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [115]:
walmart_coordinates = pd.read_csv("/Users/akhilpenmatcha/Desktop/DE-Project/weather-project/walmart_weather_df_output.csv")
walmart_coordinates.head()

,Store,Address,City,State,Zip Code,Phone,Latitude,Longitude,Temperature,Is Day or Night,Rain,Snowfall,Weather Code,Wind Speed,Wind Direction
0,2070,3101 A ST,ANCHORAGE,AK,99503-4050,(907) 563-5900,61.192242,-149.879650,-2.0,Night,0.0,6.72,71.0,2.3,288.0
1,2071,8900 OLD SEWARD HWY,ANCHORAGE,AK,99515-2022,(907) 344-5300,61.140050,-149.871154,-2.0,Night,0.0,6.72,71.0,2.3,288.0
2,4359,7405 DEBARR ROAD,ANCHORAGE,AK,99504-0000,(907) 339-9039,61.212165,-149.743372,-1.9,Night,0.0,4.76,71.0,5.2,236.0
3,2188,18600 EAGLE RIVER RD,EAGLE RIVER,AK,99577-8688,(907) 694-9780,61.309083,-149.535545,-2.5,Night,0.0,4.76,71.0,5.2,236.0
4,2722,537 JOHANSEN EXPY,FAIRBANKS,AK,99701-3165,(907) 451-9900,64.858595,-147.690644,-7.4,Night,0.0,0.35,71.0,8.6,22.0


In [70]:
walmart_coordinates= walmart_coordinates.drop(['Temperature', 'Is Day or Night', 'Rain', 'Snowfall', 'Weather Code', 'Wind Speed', 'Wind Speed','Wind Direction'], axis=1)
walmart_coordinates.head()

,Store,Address,City,State,Zip Code,Phone,Latitude,Longitude
0,2070,3101 A ST,ANCHORAGE,AK,99503-4050,(907) 563-5900,61.192242,-149.879650
1,2071,8900 OLD SEWARD HWY,ANCHORAGE,AK,99515-2022,(907) 344-5300,61.140050,-149.871154
2,4359,7405 DEBARR ROAD,ANCHORAGE,AK,99504-0000,(907) 339-9039,61.212165,-149.743372
3,2188,18600 EAGLE RIVER RD,EAGLE RIVER,AK,99577-8688,(907) 694-9780,61.309083,-149.535545
4,2722,537 JOHANSEN EXPY,FAIRBANKS,AK,99701-3165,(907) 451-9900,64.858595,-147.690644


In [71]:
walmart_coordinates.count()

Store        4434
Address      4434
City         4434
State        4434
Zip Code     4434
Phone        4434
Latitude     4425
Longitude    4425
dtype: int64

In [73]:
walmart_coordinates=walmart_coordinates.dropna(subset=['Latitude', 'Longitude'])
empty_rows1 = walmart_coordinates[walmart_coordinates[['Latitude', 'Longitude']].isnull().any(axis=1)]
print(empty_rows1)

Empty DataFrame
Columns: [Store, Address, City, State, Zip Code, Phone, Latitude, Longitude]
Index: []


In [76]:
def get_future_weather_data(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m,precipitation,rain,snowfall,wind_speed_10m,wind_direction_10m,is_day",
        "timezone": "auto"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for lat: {latitude}, lon: {longitude}")
        return None

# Fetch and process weather data for each Walmart location
weather_data_list1 = []
for _, row in walmart_coordinates.iterrows():
    # Fetch weather data using the latitude and longitude of each store
    weather_data = get_future_weather_data(row['Latitude'], row['Longitude'])
    
    if weather_data:
        # Extract necessary fields (you may need to adjust depending on your model’s requirements)
        latest_data = weather_data['hourly']
        features = {
            "Store": row['Store'],
            "Temperature": latest_data['temperature_2m'][-1],       # Last available forecasted temperature
            "Rain": latest_data['rain'][-1],                        # Last available forecasted rain
            "Snowfall": latest_data['snowfall'][-1],                # Last available forecasted snowfall
            "Wind Speed": latest_data['wind_speed_10m'][-1],        # Last available forecasted wind speed
            "Wind Direction": latest_data['wind_direction_10m'][-1],# Last available forecasted wind direction
            "Is Day or Night": "Day" if latest_data['is_day'][-1] == 1 else "Night"  # Convert is_day to Day/Night
        }
        weather_data_list1.append(features)



      Store  Temperature  Rain  Snowfall  Wind Speed  Wind Direction  \
0      2070          4.8   0.0       0.0         6.9              47   
1      2071          4.8   0.0       0.0         6.9              47   
2      4359          2.3   0.0       0.0         9.1               7   
3      2188          1.7   0.0       0.0         9.1               7   
4      2722         -1.6   0.0       0.0         8.6              68   
...     ...          ...   ...       ...         ...             ...   
4420   1412         -6.2   0.0       0.0        13.1             171   
4421   4471         -5.4   0.0       0.0         5.9             232   
4422   1457         -3.7   0.0       0.0         9.7             225   
4423   1461         -4.6   0.0       0.0         5.4               8   
4424   1508         -2.4   0.0       0.0         2.5             172   

     Is Day or Night  
0              Night  
1              Night  
2              Night  
3              Night  
4              Night

In [113]:
# Convert the list of dictionaries to a DataFrame
future_weather_df = pd.DataFrame(weather_data_list1)

# Display the resulting future weather data
print(future_weather_df)

      Store  Temperature  Rain  Snowfall  Wind Speed  Wind Direction  \
0      2070          4.8   0.0       0.0         6.9              47   
1      2071          4.8   0.0       0.0         6.9              47   
2      4359          2.3   0.0       0.0         9.1               7   
3      2188          1.7   0.0       0.0         9.1               7   
4      2722         -1.6   0.0       0.0         8.6              68   
...     ...          ...   ...       ...         ...             ...   
4420   1412         -6.2   0.0       0.0        13.1             171   
4421   4471         -5.4   0.0       0.0         5.9             232   
4422   1457         -3.7   0.0       0.0         9.7             225   
4423   1461         -4.6   0.0       0.0         5.4               8   
4424   1508         -2.4   0.0       0.0         2.5             172   

     Is Day or Night  
0              Night  
1              Night  
2              Night  
3              Night  
4              Night

In [101]:
# future_weather_df=future_weather_df.drop(['Impact Level', 'Alerts'], axis=1)
# future_weather_df.head()

,Store,Temperature,Rain,Snowfall,Wind Speed,Wind Direction,Is Day or Night
0,2070,4.8,0.0,0.0,6.9,47,0
1,2071,4.8,0.0,0.0,6.9,47,0
2,4359,2.3,0.0,0.0,9.1,7,0
3,2188,1.7,0.0,0.0,9.1,7,0
4,2722,-1.6,0.0,0.0,8.6,68,0


In [111]:
# Load the multi-output model
multi_output_model = joblib.load('multi_output_rf_model.pkl')

# Preprocess the new DataFrame
new_df = future_weather_df.copy()
new_df['Is Day or Night'] = label_encoder.fit_transform(new_df['Is Day or Night'])

# Prepare features for prediction
X_new = new_df[['Temperature', 'Rain', 'Snowfall', 'Wind Speed', 'Wind Direction', 'Is Day or Night']]

# Predict both 'Impact Level' and 'Alert' using the multi-output model
y_pred_new = multi_output_model.predict(X_new)
new_df['Predicted Impact Level'] = y_pred_new[:, 0]
new_df['Predicted Alert'] = y_pred_new[:, 1]

# Save the predictions to Excel
new_df.to_csv("predicted_impact_alert_levels_multi_output.csv", index=False)
print("Predicted impact levels and alerts saved to 'predicted_impact_alert_levels_multi_output.csv'")

Predicted impact levels and alerts saved to 'predicted_impact_alert_levels_multi_output.xlsx'
